In [64]:
%load_ext autoreload
%autoreload 2

import numpy as np

from vqf import get_classical_energy, get_pauli_str, get_cost_hamiltonian

from qiskit.utils import algorithm_globals
from qiskit.algorithms.optimizers import COBYLA
from qiskit import Aer, transpile
from qiskit.algorithms import QAOA

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Consider the factoring of the biprime

$$
m = p * q
$$

where, 

$$
m = \sum_{k=0}^{n_m -1} 2^k m_k \;\;\;, m_k \in \{0,1 \}
$$ 

$$
p = \sum_{k=0}^{n_p -1} 2^k p_k \;\;\;, p_k \in \{0,1 \}
$$ 

$$
q = \sum_{k=0}^{n_q -1} 2^k q_k \;\;\;, q_k \in \{0,1 \}
$$ 

We assume that 

$$
p \geq q
$$

$$
n_p = n_m
$$

$$
n_q = \left \lceil \frac{n_m}{2} \right \rceil
$$

Then,

$$
n_c = n_p + n_q - 1
$$

In [65]:
p = 43
q = 37
m = p * q

The set of clauses for the factoring problem 
$$
C_i = \sum _{j = 0} ^ i q_j p_{i-j} + \sum _{j = 0} ^ i z_{j,i} - m_i - \sum _{j = 0} ^ i 2^j z_{i, i+j} \;\;\;, 0 \leq i < n_c
$$

Hence, the classical energy is defined as

$$
E = \sum_{i=0} ^ {n_c} C_i^2
$$

In [66]:
energy = get_classical_energy(m)
energy

p_0**2*q_0**2 + p_0**2*q_1**2 + p_0**2*q_2**2 + p_0**2*q_3**2 + p_0**2*q_4**2 + p_0**2*q_5**2 + 2*p_0*p_1*q_0*q_1 + 2*p_0*p_1*q_1*q_2 + 2*p_0*p_1*q_2*q_3 + 2*p_0*p_1*q_3*q_4 + 2*p_0*p_1*q_4*q_5 + 2*p_0*p_2*q_0*q_2 + 2*p_0*p_2*q_1*q_3 + 2*p_0*p_2*q_2*q_4 + 2*p_0*p_2*q_3*q_5 + 2*p_0*p_3*q_0*q_3 + 2*p_0*p_3*q_1*q_4 + 2*p_0*p_3*q_2*q_5 + 2*p_0*p_4*q_0*q_4 + 2*p_0*p_4*q_1*q_5 + 2*p_0*p_5*q_0*q_5 - 2*p_0*q_0 - 1024*p_0*q_1*z_1_10 - 4*p_0*q_1*z_1_2 - 8*p_0*q_1*z_1_3 - 16*p_0*q_1*z_1_4 - 32*p_0*q_1*z_1_5 - 64*p_0*q_1*z_1_6 - 128*p_0*q_1*z_1_7 - 256*p_0*q_1*z_1_8 - 512*p_0*q_1*z_1_9 - 2*p_0*q_1 + 2*p_0*q_2*z_1_2 - 512*p_0*q_2*z_2_10 - 4*p_0*q_2*z_2_3 - 8*p_0*q_2*z_2_4 - 16*p_0*q_2*z_2_5 - 32*p_0*q_2*z_2_6 - 64*p_0*q_2*z_2_7 - 128*p_0*q_2*z_2_8 - 256*p_0*q_2*z_2_9 - 2*p_0*q_2 + 2*p_0*q_3*z_1_3 + 2*p_0*q_3*z_2_3 - 256*p_0*q_3*z_3_10 - 4*p_0*q_3*z_3_4 - 8*p_0*q_3*z_3_5 - 16*p_0*q_3*z_3_6 - 32*p_0*q_3*z_3_7 - 64*p_0*q_3*z_3_8 - 128*p_0*q_3*z_3_9 + 2*p_0*q_4*z_1_4 + 2*p_0*q_4*z_2_4 + 2*p_0*q_4*z_3_4

We apply the following classical preprocessing rules

$$
x y - 1 = 0 \Rightarrow x = y = 1,
$$

$$
x + y − 1 = 0 \Rightarrow xy = 0,
$$

$$
a − bx = 0 \Rightarrow x = 1,
$$

$$
\sum_i x_i = 0 \Rightarrow x_i = 0,
$$

$$
\sum_{i=1}^a x_i − a = 0 \Rightarrow x_i = 1.
$$

to the classical energy function to get the simplified energy function

$$
E^{'} = \sum_{i=0} ^ {n_c} C_i^{'2}
$$

In [67]:
energy = get_classical_energy(m, apply_rules=True, verbose=True)
energy

Preprocessing iteration: 0
Current clause 1 : p_1 + q_1 - 1
Rule 2 applied! p_1 = 1 - q_1
Current clause 2 : p_2 + q_2 - 2*z_2_3 - 1
Z rule 1 applied! z_2_3 = 0
Rule 2 applied! q_2 = 1 - p_2
Current clause 3 : p_3 - 2*q_1*q_2 + q_1 + q_2 + q_3 - 2*z_3_4 - 4*z_3_5
Current clause 4 : p_3*q_1 + p_4 - q_1*q_3 + q_3 + q_4 + z_3_4 - 2*z_4_5 - 4*z_4_6 - 1
Current clause 5 : p_3*q_2 + p_4*q_1 + p_5 - q_1*q_4 - q_2*q_3 + q_3 + q_4 + q_5 + z_3_5 + z_4_5 - 2*z_5_6 - 4*z_5_7 - 1
Current clause 6 : p_3*q_3 + p_4*q_2 + p_5*q_1 + p_6 - q_1*q_5 - q_2*q_4 + q_4 + q_5 + z_4_6 + z_5_6 - 2*z_6_7 - 4*z_6_8 - 8*z_6_9
Current clause 7 : p_3*q_4 + p_4*q_3 + p_5*q_2 + p_6*q_1 + p_7 - q_2*q_5 + q_5 + z_5_7 + z_6_7 - 8*z_7_10 - 2*z_7_8 - 4*z_7_9
Current clause 8 : p_3*q_5 + p_4*q_4 + p_5*q_3 + p_6*q_2 + p_7*q_1 + p_8 + z_6_8 + z_7_8 - 4*z_8_10 - 2*z_8_9
Current clause 9 : p_4*q_5 + p_5*q_4 + p_6*q_3 + p_7*q_2 + p_8*q_1 + p_9 + z_6_9 + z_7_9 + z_8_9 - 2*z_9_10 - 1
Current clause 10 : p_10 + p_5*q_5 + p_6*q_4 + p_

2*p_10*p_5*q_5 + 2*p_10*p_6*q_1*q_5 + 2*p_10*p_6*q_4 + 2*p_10*p_7*q_1*q_4 + 2*p_10*p_7*q_2*q_5 + 2*p_10*p_7*q_3 + 2*p_10*p_8*q_1*q_3 + 2*p_10*p_8*q_2*q_4 + 2*p_10*p_8*q_2 + 2*p_10*p_8*q_3*q_5 + 2*p_10*p_9*q_1*q_2 + 2*p_10*p_9*q_1 + 2*p_10*p_9*q_2*q_3 + 2*p_10*p_9*q_3*q_4 + 2*p_10*p_9*q_4*q_5 + p_10*q_1 + p_10*q_2 + p_10*q_3 + p_10*q_4 + p_10*q_5 + 2*p_10*z_7_10 + 2*p_10*z_8_10 + 2*p_10*z_9_10 - p_10 + 2*p_3*p_4*q_1*q_2 + 2*p_3*p_4*q_1 + 2*p_3*p_4*q_2*q_3 + 2*p_3*p_4*q_3*q_4 + 2*p_3*p_4*q_4*q_5 + 2*p_3*p_5*q_1*q_3 + 2*p_3*p_5*q_2*q_4 + 2*p_3*p_5*q_2 + 2*p_3*p_5*q_3*q_5 + 2*p_3*p_6*q_1*q_4 + 2*p_3*p_6*q_2*q_5 + 2*p_3*p_6*q_3 + 2*p_3*p_7*q_1*q_5 + 2*p_3*p_7*q_4 + 2*p_3*p_8*q_5 - 2*p_3*q_1*q_2*q_4 - 4*p_3*q_1*q_2 - 2*p_3*q_1*q_3*q_5 + 2*p_3*q_1*q_4 + 2*p_3*q_1*z_3_4 - 4*p_3*q_1*z_4_5 - 8*p_3*q_1*z_4_6 + p_3*q_1 - 2*p_3*q_2*q_3*q_4 - 2*p_3*q_2*q_4*q_5 + 2*p_3*q_2*q_4 + 2*p_3*q_2*q_5 + 2*p_3*q_2*z_3_5 + 2*p_3*q_2*z_4_5 - 4*p_3*q_2*z_5_6 - 8*p_3*q_2*z_5_7 + p_3*q_2 + 2*p_3*q_3*q_4 + 2*p_3*q_3

We make the ising substitution for each term of the classical energy $E^{'}$

$$
b_k \rightarrow \frac{1}{2} \left( 1 - \sigma_{b, k}^z \right)
$$

In [68]:
bits = energy.free_symbols
n_bits = len(bits)
print(f"The total number of classical bits: {n_bits}")

The total number of classical bits: 28


In [69]:
for a in range(len(energy.args)):
    term = energy.args[a]
    print(term)
    ising = get_pauli_str(term, bits)
    print(str(ising) + "\n")

4
None

p_3
0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
- 0.5 * IZIIIIIIIIIIIIIIIIIIIIIIIIII

p_6
0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
- 0.5 * IIIIIIIIIIIIIIIIIIIIIIIIZIII

p_7
0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
- 0.5 * IIIIIIIIIZIIIIIIIIIIIIIIIIII

p_8
0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
- 0.5 * IIIIIZIIIIIIIIIIIIIIIIIIIIII

q_1
0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
- 0.5 * IIIZIIIIIIIIIIIIIIIIIIIIIIII

q_2
0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
- 0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIZII

q_5
0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
- 0.5 * IIIIZIIIIIIIIIIIIIIIIIIIIIII

-p_10
-0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIIIIIIIIIIIIIZIIIIIIIIIIII

-p_4
-0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIIIIIIIIIIIIIIIIIIIIIZIIII

-p_5
-0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIIIIZIIIIIIIIIIIIIIIIIIIII

-p_9
-0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIIIIIIIIIIIIIIIIIIIZIIIIII

-q_3
-0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIIIIIIIIIIIIIIIIIIIIZIIIII

-q_4
-0.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
+ 0.5 * IIIIIIII

By summing those terms we get the factoring cost hamiltonian 

$$
H = \sum_{i=0} ^ {n_c} C_i^{'2}
$$

In [70]:
H = get_cost_hamiltonian(m)
print(str(H))

127.125 * IIIIIIIIIIIIIIIIIIIIIIIIIIII
- 3.375 * IIIIIIIIIIIIIIIIIIIIIIIIIZII
+ 2.375 * IIIIIIIIIIIIIIIIZIIIIIIIIIII
+ 2.75 * IIIIIIIIIIIIIIIIZIIIIIIIIZII
+ 0.75 * IIIIZIIIIIIIIIIIIIIIIIIIIIII
+ 2.75 * IIIIZIIIIIIIIIIIIIIIIIIIIZII
+ 1.375 * IIIIZIIIIIIIIIIIZIIIIIIIIIII
+ 0.25 * IIIIZIIIIIIIIIIIZIIIIIIIIZII
- 0.75 * IIIZIIIIIIIIIIIIIIIIIIIIIIII
+ 2.0 * IIIZIIIIIIIIIIIIIIIIIIIIIZII
+ 1.125 * IIIZIIIIIIIIIIIIZIIIIIIIIIII
+ 2.75 * IIIZZIIIIIIIIIIIIIIIIIIIIIII
+ 0.125 * IIIZZIIIIIIIIIIIIIIIIIIIIZII
+ 0.375 * IIIZZIIIIIIIIIIIZIIIIIIIIIII
+ 0.125 * IIIZZIIIIIIIIIIIZIIIIIIIIZII
+ 3.125 * IIIIIIIIIIIIIIIIIIIIIIZIIIII
+ 1.125 * IIIIIIIIIIIIIIIIIIIIIIZIIZII
+ 1.375 * IIIIIIIIIIIIIIIIZIIIIIZIIIII
+ 0.25 * IIIIIIIIIIIIIIIIZIIIIIZIIZII
+ 1.625 * IIIZIIIIIIIIIIIIIIIIIIZIIIII
+ 0.5 * IIIZIIIIIIIIIIIIIIIIIIZIIZII
+ 0.375 * IIIZIIIIIIIIIIIIZIIIIIZIIIII
+ 0.125 * IIIZIIIIIIIIIIIIZIIIIIZIIZII
- 4.75 * IIIIIIIIIIIIIIIIIIIIIZIIIIII
+ 1.25 * IIIIIIIIIIIIIIIIZIIIIZIIIIII
- 3.625 * IIIIIZIIIIIIIIIIIIIIIIIIIIII

In [71]:
algorithm_globals.random_seed = 10598
optimizer = COBYLA()
instance = Aer.get_backend("aer_simulator")
qaoa = QAOA(optimizer, quantum_instance=instance)
# result = qaoa.compute_minimum_eigenvalue(H)

### Before transpilation

In [72]:
params = np.ones(2)  # a typical QAOA ansatz has two parameters, we set them to be ones.
qc = qaoa.construct_circuit(params, H)
qc = qc[0]
# qc = qc.decompose().decompose()#.decompose()
# qc.draw('mpl')

In [73]:
n_qubits = qc.num_qubits
depth = qc.depth()
gates = qc.count_ops()
# n_cnots = gates['cx']
# n_u3 = gates['u3']
# n_gates = n_cnots + n_u3

print(
    f"The circuit parameters for factoring the biprime {m} using QAOA (no transpilation) \n"
)
print(f"The depth of the circuit: {depth}")
print(f"The total number of qubits: {n_qubits}")
print(f"The total number of gates: {str(gates)}")
# print(f"The total number of U gates: {n_u3}")
# print(f"The total number of CX gates: {n_cnots}")

The circuit parameters for factoring the biprime 1591 using QAOA (no transpilation) 

The depth of the circuit: 3
The total number of qubits: 28
The total number of gates: OrderedDict([('h', 28), ('PauliEvolution', 2)])


In [74]:
for level in range(4):
    t_circ = transpile(qc, basis_gates=["cx", "u3"], optimization_level=level)
    n_qubits = t_circ.num_qubits
    depth = t_circ.depth()
    gates = t_circ.count_ops()
    n_cnots = gates["cx"]
    n_u3 = gates["u3"]
    n_gates = n_cnots + n_u3

    print(
        f"The circuit parameters for factoring the biprime {m} using QAOA  (transpilation level: {level}) \n"
    )
    print(f"The depth of the circuit: {depth}")
    print(f"The total number of qubits: {n_qubits}")
    print(f"The total number of gates: {n_gates}")
    print(f"The total number of U gates: {n_u3}")
    print(f"The total number of CX gates: {n_cnots}")
    print("--------------------------------------------")

The circuit parameters for factoring the biprime 1591 using QAOA  (transpilation level: 0) 

The depth of the circuit: 2152
The total number of qubits: 28
The total number of gates: 2923
The total number of U gates: 729
The total number of CX gates: 2194
--------------------------------------------
The circuit parameters for factoring the biprime 1591 using QAOA  (transpilation level: 1) 

The depth of the circuit: 1887
The total number of qubits: 28
The total number of gates: 2625
The total number of U gates: 701
The total number of CX gates: 1924
--------------------------------------------
The circuit parameters for factoring the biprime 1591 using QAOA  (transpilation level: 2) 

The depth of the circuit: 1887
The total number of qubits: 28
The total number of gates: 2625
The total number of U gates: 701
The total number of CX gates: 1924
--------------------------------------------
The circuit parameters for factoring the biprime 1591 using QAOA  (transpilation level: 3) 

The dep